### Libraries

In [2]:
# General Imports
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import sys
import yaml
import numpy as np
from glob import glob
import time
import cv2

# Tensorflow Imports
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Activation, Dense, Lambda, Input, Dense
from tensorflow.keras.layers import MaxPooling2D, Flatten, Reshape, Concatenate
from tensorflow.keras.layers import SeparableConv2D, Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras.utils import Sequence

disable_eager_execution()
print(f"TensorFlow version: {tf.__version__}")

# Local Module Imports
sys.path.append("../src")  # adds source code directory
from utils import frame_to_label
from utils import frames_to_video, save_history
from visualization import plot_learning_curves
from polygon_handle import masks_to_polygons
from log_setup import logger

TensorFlow version: 2.15.0


### Global Variables

In [3]:
""" 
DATA: "full" (full dataset), "sampled" (distance sampled dataset) 
        or "unet" (unet generated dataset)
MODE: "interpol" (interpolation) or "extrapol" (extrapolation)
MODEL: "CVAE"
PERCENTAGE: percentage of training data to be used for training
LAST_FRAME: last frame number of the video
"""

DATA = "full"
MODE = "extrapol"
MODEL = "CVAE"
PERCENTAGE = 50
LAST_FRAME = 22500

### Directories

In [4]:
BASE_DIR = os.path.dirname(os.getcwd())
dataset_dir = os.path.join(BASE_DIR, "dataset")
data_dir = os.path.join(BASE_DIR, "data")
config_file = os.path.join(BASE_DIR, "config.yml")

# Output PNG directory
if MODE == "extrapol":
    output_dir = os.path.join(BASE_DIR, "outputs", "CVAE", MODE, str(PERCENTAGE), DATA)
    logger.info(
        f"Data: {DATA}, Mode: {MODE}, Model: {MODEL} Percentage: {PERCENTAGE}%,\nOutput directory: {output_dir}"
    )
elif MODE == "interpol":
    output_dir = os.path.join(BASE_DIR, "outputs", "CVAE", MODE, DATA)
    logger.info(
        f"\nData: {DATA}, Mode: {MODE}, Model: {MODEL}\nOutput directory: {output_dir}"
    )

INFO - Data: full, Mode: extrapol, Model: CVAE Percentage: 50%,
Output directory: /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/50/full


### Config file

In [5]:
with open(config_file, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

### Data loading

In [6]:
# Training data
if DATA == "full":
    train_dir = os.path.join(BASE_DIR, config["data"]["full"]["train_dir"], "masks")
    # sort the paths
    train_paths = sorted(glob(os.path.join(train_dir, "*.png")))
    # extract labels from the paths
    train_labels = [
        int(os.path.basename(m).split("_")[1].split(".")[0]) * 100 for m in train_paths
    ]
    epochs = config["CVAE"]["epochs"]

elif DATA == "sampled":
    sampled_masks_txt_path = os.path.join(BASE_DIR, config["data"]["sampled_masks_txt"])
    with open(sampled_masks_txt_path, "r", encoding="utf-8") as f:
        polygons = f.readlines()
        # extract indexes
    indexes = [int(polygon.split(",")[0]) for polygon in polygons]
    train_dir = os.path.join(BASE_DIR, config["data"]["sampled"]["train_dir"], "masks")
    train_paths = sorted(glob(os.path.join(train_dir, "*.png")))
    train_labels = [100 * i for i in indexes]
    epochs = config["CVAE"]["epochs"]

elif DATA == "unet":
    train_dir = os.path.join(BASE_DIR, config["data"]["unet"]["train_dir"], "masks")
    train_paths = sorted(glob(os.path.join(train_dir, "*.png")))
    train_labels = [
        int(os.path.basename(m).split("_")[1].split(".")[0]) for m in train_paths
    ]
    epochs = 2


# Test data
test_dir = os.path.join(BASE_DIR, config["data"]["test"]["test_dir"], "masks")
test_paths = sorted(glob(os.path.join(test_dir, "*.png")))
test_labels = [
    int(os.path.basename(m).split("_")[1].split(".")[0]) * 100 + 20250
    for m in test_paths
]

if MODE == "extrapol":
    # Truncate the training data
    train_paths = train_paths[: int(len(train_paths) * PERCENTAGE / 100)]
    train_labels = train_labels[: int(len(train_labels) * PERCENTAGE / 100)]
    logger.info(f"No. train. samples: {len(train_paths)} out of {LAST_FRAME} ({PERCENTAGE}%) | No. test samples: {len(test_paths)}")
elif MODE == "interpol":
    logger.info(f"No. train. samples: {len(train_paths)} out of {LAST_FRAME} | No. test samples: {len(test_paths)}")


INFO - No. train. samples: 113 out of 22500 (50%) | No. test samples: 23


In [7]:
class CVAEDataGenerator(Sequence):
    """
    A data generator for the Conditional Variational Autoencoder (CVAE) model.

    This class generates batches of images and corresponding labels from a given set of data paths and labels.
    It shuffles the data at the end of each epoch to ensure that the model sees all data in each epoch.

    Attributes:
        data_paths: A list of paths to the data files.
        labels: A list of corresponding labels for the data files.
        batch_size: The number of samples per gradient update.
        input_shape: The shape of the input data.
        num_frames: The total number of frames in the data.

    Methods:
        __init__: Initializes the data generator.
        __len__: Returns the number of batches in the data.
        __getitem__: Returns a batch of images and labels.
        on_epoch_end: Shuffles the data at the end of each epoch.
        load_and_preprocess_data: Loads and preprocesses a batch of images and labels.
        load_preprocess_mask: Loads and preprocesses a single mask image.
    """

    def __init__(self, data_paths, labels, batch_size, input_shape, last_frame):
        """
        Initializes the data generator.

        Args:
            data_paths: A list of paths to the data files.
            labels: A list of corresponding labels for the data files.
            batch_size: The number of samples per gradient update.
            input_shape: The shape of the input data.
            last_frame: The total number of frames in the data.
        """
        self.data_paths = data_paths
        self.labels = labels
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.num_frames = last_frame
        self.on_epoch_end()

    def __len__(self):
        """
        Returns the number of batches in the data.

        Returns:
            The number of batches in the data.
        """
        return int(np.ceil(len(self.data_paths) / self.batch_size))

    def __getitem__(self, index):
        """
        Returns a batch of images and labels.

        Args:
            index: The index of the batch.

        Returns:
            A batch of images and labels.
        """
        start_idx = index * self.batch_size
        end_idx = (index + 1) * self.batch_size
        batch_data_paths = self.data_paths[start_idx:end_idx]
        batch_labels = self.labels[start_idx:end_idx]

        batch_images, batch_labels = self.load_and_preprocess_data(
            batch_data_paths, batch_labels
        )
        return [batch_images, batch_labels], batch_images

    def on_epoch_end(self):
        """
        Shuffles the data at the end of each epoch.
        """
        indices = np.arange(len(self.data_paths))
        np.random.shuffle(indices)
        self.data_paths = [self.data_paths[i] for i in indices]
        self.labels = [self.labels[i] for i in indices]

    def load_and_preprocess_data(self, batch_data_paths, batch_labels):
        """
        Loads and preprocesses a batch of images and labels.

        Args:
            batch_data_paths: A list of paths to the data files.
            batch_labels: A list of corresponding labels for the data files.

        Returns:
            A batch of images and labels.
        """
        batch_images = []
        batch_labels_processed = []
        for data_path, label in zip(batch_data_paths, batch_labels):
            image, label = self.load_preprocess_mask(
                data_path, label, self.input_shape, self.num_frames
            )
            batch_images.append(image)
            batch_labels_processed.append(label)
        return np.array(batch_images), np.array(batch_labels_processed)

    def load_preprocess_mask(self, mask_path, label, output_dims, last_frame):
        """
        Loads and preprocesses a single mask image.

        Args:
            mask_path: The path to the mask file.
            label: The corresponding label for the mask file.
            output_dims: The desired dimensions of the mask.
            last_frame: The total number of frames in the data.

        Returns:
            A preprocessed mask image and its corresponding label.
        """
        # Check if the file exists
        if not os.path.exists(mask_path):
            raise FileNotFoundError(f"No such file: '{mask_path}'")

        # Read and decode the image
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Resize the image
        mask = cv2.resize(mask, output_dims)

        # Add channel dimension
        mask = np.expand_dims(mask, axis=-1)

        # Normalize the mask
        mask = (mask / 127.5) - 1

        # Normalize the label
        label = label / last_frame
        label = np.expand_dims(label, axis=-1)

        return mask, label


input_shape = config["CVAE"]["input_shape"]

# Create training data generator
train_data_gen = CVAEDataGenerator(
    data_paths=train_paths,
    labels=train_labels,
    batch_size=1,
    input_shape=input_shape[:2],
    last_frame=LAST_FRAME,
)

# Create testing data generator
test_data_gen = CVAEDataGenerator(
    data_paths=test_paths,
    labels=test_labels,
    batch_size=1,
    input_shape=input_shape[:2],
    last_frame=LAST_FRAME,
)

### C-VAE definition

In [8]:
def deconv_block(input, filters, f_init="he_normal"):
    """
    Apply two convolutional layers with ReLU activation function.

    Args:
        input (tensor): Input tensor to the block.
        filters (int): Number of filters in the convolutional layers.

    Returns:
        tensor: Output tensor of the block with ReLU activation.
    """
    x = Conv2DTranspose(
        filters,
        kernel_size=(4, 4),
        strides=2,
        kernel_initializer=f_init,
        data_format="channels_last",
        padding="same",
    )(input)

    x = SeparableConv2D(
        filters,
        kernel_size=(4, 4),
        depthwise_initializer=f_init,
        pointwise_initializer=f_init,
        padding="same",
    )(x)
    x = Activation(tf.nn.leaky_relu)(x)

    x = SeparableConv2D(
        filters,
        kernel_size=(4, 4),
        depthwise_initializer=f_init,
        pointwise_initializer=f_init,
        padding="same",
    )(x)
    activation = Activation(tf.nn.leaky_relu)(x)

    return activation


def conv_block(input, filters, f_init="he_normal"):
    """
    Apply two convolutional layers with ReLU activation function.

    Args:
        input (tensor): Input tensor to the block.
        filters (int): Number of filters in the convolutional layers.

    Returns:
        tensor: Output tensor of the block with ReLU activation.
    """
    x = SeparableConv2D(
        filters,
        kernel_size=(4, 4),
        depthwise_initializer=f_init,
        pointwise_initializer=f_init,
        padding="same",
    )(input)
    x = Activation(tf.nn.leaky_relu)(x)

    x = SeparableConv2D(
        filters,
        kernel_size=(4, 4),
        depthwise_initializer=f_init,
        pointwise_initializer=f_init,
        padding="same",
    )(x)
    ativ = Activation(tf.nn.leaky_relu)(x)

    m_pool = MaxPooling2D(
        pool_size=(2, 2), strides=2, data_format="channels_last", padding="same"
    )(ativ)

    return m_pool


def sampler(args):
    """
    Reparameterization trick by sampling fr an isotropic unit Gaussian.

    Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def mse_kl_loss(y_true, y_pred, beta: float = 1.0):
    """Calculate loss = reconstruction loss + KL loss for each data in minibatch"""
    # E[log P(X|z)]
    squared_difference = tf.square(y_true - y_pred)
    reconstruction = tf.reduce_mean(squared_difference, axis=-1)
    # D_KL(Q(z|X) || P(z|X)); calculate in closed from as both dist. are Gaussian
    kl_divergence = 0.5 * tf.reduce_sum(
        tf.exp(z_log_var) + tf.square(z_mean) - 1.0 - z_log_var, axis=-1
    )
    return reconstruction + beta * kl_divergence

In [9]:
H, W, C = config["CVAE"]["input_shape"]

# --------
# Encoder
# --------

encoder_inputs = Input(shape=(H, W, C))
# Reshape input to 2D image

x = conv_block(
    encoder_inputs, config["CVAE"]["ref_filters"] * 2, config["CVAE"]["w_init"]
)
x = conv_block(x, config["CVAE"]["ref_filters"] * 1, config["CVAE"]["w_init"])
x = Flatten()(x)
x = Dense(64, activation="leaky_relu")(x)

# VAE specific layers for mean and log variance
z_mean = Dense(config["CVAE"]["latent_dim"], activation="leaky_relu", name="z_mean")(x)
z_log_var = Dense(
    config["CVAE"]["latent_dim"], activation="leaky_relu", name="z_log_var"
)(x)

# Sampling layer to sample z from the latent space
z = Lambda(sampler, output_shape=(config["CVAE"]["latent_dim"],), name="z")(
    [z_mean, z_log_var]
)

# Instantiate encoder model
encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

# --------
# Decoder
# --------

latent_inputs = Input(shape=(config["CVAE"]["latent_dim"],), name="z_sampling")
label_size = 1 # one tf.float32 label
label_inputs = Input(shape=(label_size,), name="label")
decoder_inputs = Concatenate()([latent_inputs, label_inputs])
x = Dense(64 * 64 * 64, activation="leaky_relu")(decoder_inputs)
x = Reshape((128, 128, 16))(x)
x = deconv_block(x, config["CVAE"]["ref_filters"] * 2, config["CVAE"]["w_init"])
x = deconv_block(x, config["CVAE"]["ref_filters"] * 4, config["CVAE"]["w_init"])
decoder_output = Conv2DTranspose(1, 3, activation="tanh", padding="same")(x)

decoder = Model([latent_inputs, label_inputs], decoder_output, name="decoder")

# -----------------
# Conditional VAE
# -----------------

outputs = decoder([encoder(encoder_inputs)[2], label_inputs])
cvae = Model([encoder_inputs, label_inputs], outputs, name="cvae")
cvae.summary()

Model: "cvae"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 encoder (Functional)        [(None, 64),                 3357281   ['input_1[0][0]']             
                              (None, 64),                 6                                       
                              (None, 64)]                                                         
                                                                                                  
 label (InputLayer)          [(None, 1)]                  0         []                            
                                                                                               

### Callbacks

In [10]:
class ReduceLROnPlateauSteps(tf.keras.callbacks.ReduceLROnPlateau):
    def __init__(
        self, monitor="val_loss", factor=0.5, patience=500, min_lr=1e-8, **kwargs
    ):
        super().__init__(
            monitor=monitor, factor=factor, patience=patience, min_lr=min_lr, **kwargs
        )
        self.wait = 0
        self.best = 0

    def on_train_batch_end(self, batch, logs=None):
        current = self.get_monitor_value(logs)
        if current < self.best:
            self.best = current
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.wait = 0
                new_lr = self.model.optimizer.learning_rate * self.factor
                new_lr = tf.keras.backend.get_value(new_lr)
                self.model.optimizer.learning_rate = new_lr
                print("Reducing learning rate to %s." % (new_lr,))

    def get_monitor_value(self, logs):
        logs = logs or {}
        monitor_value = logs.get(self.monitor)
        if monitor_value is None:
            logger.warning(
                "Learning rate reduction on plateau conditioned on metric `%s` "
                "which is not available. Available metrics are: %s"
                % (self.monitor, ", ".join(list(logs.keys()))),
                RuntimeWarning,
            )
        return monitor_value


class EarlyStoppingSteps(tf.keras.callbacks.EarlyStopping):
    def __init__(self, monitor="val_loss", min_delta=0, patience=500, **kwargs):
        super().__init__(
            monitor=monitor, min_delta=min_delta, patience=patience, **kwargs
        )
        self.wait = 0
        self.best = 0

    def on_train_batch_end(self, batch, logs=None):
        """
        At the end of each batch, check if the monitored quantity has improved.
        If number of batches since the last improvement is more than the patience,
        stop training.
        """
        current = self.get_monitor_value(logs)
        if current < self.best:
            self.best = current
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.wait = 0
                self.stopped_epoch = self.model.history.epoch[-1]
                self.model.stop_training = True
                print("Early stopping")

    def get_monitor_value(self, logs):
        logs = logs or {}
        monitor_value = logs.get(self.monitor)
        if monitor_value is None:
            logger.warning(
                "Early stopping conditioned on metric `%s` "
                "which is not available. Available metrics are: %s"
                % (self.monitor, ", ".join(list(logs.keys()))),
                RuntimeWarning,
            )
        return monitor_value


class ModelCheckpointSteps(tf.keras.callbacks.ModelCheckpoint):
    def __init__(
        self,
        filepath,
        monitor="val_loss",
        save_best_only=False,
        save_weights_only=False,
        mode="auto",
        verbose=0,
        save_freq="epoch",
        **kwargs
    ):
        super().__init__(
            filepath=filepath,
            monitor=monitor,
            save_best_only=save_best_only,
            save_weights_only=save_weights_only,
            mode=mode,
            verbose=verbose,
            save_freq=save_freq,
            **kwargs
        )
        self.step_count = 0

    def on_train_batch_end(self, batch, logs=None):
        self.step_count += 1
        if self.step_count % self.save_freq == 0:
            self.save_weights(self.filepath, overwrite=True)


class HistoryLogger(tf.keras.callbacks.Callback):
    def __init__(self, log_interval):
        super().__init__()
        self.log_interval = log_interval
        self.step_count = 0
        self.history = []

    def on_train_batch_end(self, batch, logs=None):
        self.step_count += 1
        if self.step_count % self.log_interval == 0:
            self.history.append(logs)

    def get_history(self):
        return self.history

In [11]:
reduce_lr = ReduceLROnPlateauSteps(
    monitor="loss", factor=0.5, mode="min", patience=5000, verbose=1, min_lr=1e-8
)

early_stopping = EarlyStoppingSteps(
    monitor="loss",
    min_delta=0,
    patience=10000,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)

checkpoint_dir = os.path.join(BASE_DIR, config["data"]["checkpoint_dir"])
if MODE == "extrapol":
    checkpoint_path = os.path.join(
        checkpoint_dir, f"cvae_{DATA}_{MODE}_{PERCENTAGE}.h5"
    )
elif MODE == "interpol":
    checkpoint_path = os.path.join(checkpoint_dir, f"cvae_{DATA}_{MODE}.h5")

# use ModelCheckpoint to save best model
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor="loss",
    mode="auto",
    verbose=1,
    save_weights_only=True,
)

history_logger = HistoryLogger(log_interval=500)

### Model compilation

In [12]:
cvae.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(
        learning_rate=config["CVAE"]["learning_rate"]
    ),
    loss=mse_kl_loss,
)

### Training

In [13]:
cvae.optimizer.lr = config["CVAE"]["learning_rate"]

# Fit the model
history = cvae.fit(
    train_data_gen,
    steps_per_epoch=len(train_data_gen),
    epochs=epochs,
    validation_data=test_data_gen,
    validation_steps=len(test_data_gen),
    callbacks=[reduce_lr, early_stopping, model_checkpoint, history_logger],
)

Epoch 1/500
113/113 [==============================] - ETA: 0s - batch: 56.0000 - size: 1.0000 - loss: 0.7527

/home/tiagociic/miniconda3/envs/cvae/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



Epoch 1: loss improved from inf to 0.75274, saving model to /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/checkpoints/cvae_full_extrapol_50.h5
113/113 [==============================] - 27s 160ms/step - batch: 56.0000 - size: 1.0000 - loss: 0.7527 - val_loss: 0.9514 - lr: 3.0000e-04
Epoch 2/500
113/113 [==============================] - ETA: 0s - batch: 56.0000 - size: 1.0000 - loss: 0.3117
Epoch 2: loss improved from 0.75274 to 0.31165, saving model to /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/checkpoints/cvae_full_extrapol_50.h5
113/113 [==============================] - 18s 155ms/step - batch: 56.0000 - size: 1.0000 - loss: 0.3117 - val_loss: 0.7230 - lr: 3.0000e-04
Epoch 3/500
113/113 [==============================] - ETA: 0s - batch: 56.0000 - size: 1.0000 - loss: 0.2068
Epoch 3: loss improved from 0.31165 to 0.20677, saving model to /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/checkpoints/cvae_full_extrapol_50.h5
113/113 [=========

In [ ]:
# plot and save learning curves
if MODE == "extrapol":
    save_history(
        history, os.path.join(checkpoint_dir, f"history_{DATA}_{MODE}_{PERCENTAGE}.csv")
    )
    # plot_learning_curves(
    #     history,
    #     log_scale=True,
    #     plt_title=f"CVAE {DATA} {MODE} {PERCENTAGE}",
    #     save_fig=True,
    # )

elif MODE == "interpol":
    save_history(history, os.path.join(checkpoint_dir, f"history_{DATA}_{MODE}.csv"))
    # plot_learning_curves(
    #     history, log_scale=True, plt_title=f"CVAE_{DATA}_{MODE}", save_fig=True
    # )

ValueError: All arrays must be of the same length

### Inference

In [1]:
# load the best model
cvae.load_weights(checkpoint_path)

NameError: name 'cvae' is not defined

In [ ]:
def generate_frames(
    decoder, output_dir: str, total_frames: int = 22500, resize_original: bool = False
):
    """
    Generates and saves the frames from a trained decoder.

    Parameters:
        decoder (keras.Model): The trained decoder.
        output_dir (str): The path to the output directory.
        total_frames (int): The total number of frames to generate.
        resize_original (bool): Whether to resize the frames to the original dimensions.
    """

    start_total_time = time.time()

    frames_num = np.arange(1, total_frames + 1, 1)

    for i in range(total_frames):
        frame_num = frames_num[i]

        # Sample from the latent space
        z_sample = np.full((1, config["CVAE"]["latent_dim"]), 0.5)

        # Generate the frame
        try:
            start_time = time.time()
            reconst = decoder.predict([z_sample, frame_to_label(frame_num)])
            reconst_time = (time.time() - start_time) * 1000
            reconst = np.squeeze(reconst, axis=0)
        except Exception as e:
            print(f"Error generating frame {frame_num}: {e}")
            continue

        if resize_original:
            start_time = time.time()
            reconst = tf.image.resize(
                images=reconst, size=config["data"]["original_vid_dims"]
            )
            resize_time = (time.time() - start_time) * 1000
        else:
            resize_time = 0.0  # Not resizing

        # Binarize the reconstructed image with OpenCV
        start_time = time.time()
        _, thresh_img = cv2.threshold(
            reconst, config["CVAE"]["threshold"], 255, cv2.THRESH_BINARY
        )
        threshold_time = (time.time() - start_time) * 1000

        # Save the thresholded image as png in grayscale
        try:
            start_time = time.time()
            cv2.imwrite(
                os.path.join(output_dir, f"frame_{frame_num:06d}.png"), thresh_img
            )
            save_time = (time.time() - start_time) * 1000
        except Exception as e:
            print(f"Error saving frame {frame_num}: {e}")
            continue

        # Print progress with time information
        print(
            f"Generated frame {i+1} of {total_frames} | "
            f"Reconst: {reconst_time:.2f}ms | "
            f"Resize: {resize_time:.2f}ms | "
            f"Threshold: {threshold_time:.2f}ms | "
            f"Save: {save_time:.2f}ms | "
            f"Elapsed Time: {time.time() - start_total_time:.2f}s  ",
            end="\r",
        )
    print()

In [ ]:
output_png_dir = os.path.join(output_dir, "PNG")
generate_frames(decoder, output_png_dir, total_frames=LAST_FRAME)

/home/tiagociic/miniconda3/envs/cvae/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Generated frame 22500 of 22500 | Reconst: 24.35ms | Resize: 0.00ms | Threshold: 0.10ms | Save: 0.64ms | Elapsed Time: 548.33s  


In [ ]:
# generate video from the generated frames
if MODE == "extrapol":
    file_name = f"video_{DATA}_{MODE}_{PERCENTAGE}"
    title = f"CVAE: {MODE}ation - {DATA}, {PERCENTAGE}, {config['CVAE']['epochs']} epochs, 10x speed"
elif MODE == "interpol":
    file_name = f"video_{DATA}_{MODE}"
    title = f"CVAE: {MODE}ation - {DATA}, {config['CVAE']['epochs']} epochs, 10x speed"

frames_to_video(
    img_list_dir=os.path.join(output_dir, "PNG"),
    output_dir=output_dir,
    output_resolution=config["data"]["original_vid_dims"],
    title=title,
    f_ps=250,  # 10x speed
    file_name=file_name,
    frame_num_text=True,
    font_size=1,
)

INFO - Creating image list...                          
INFO - Writing frames to file 1/22500
INFO - Writing frames to file 1001/22500
INFO - Writing frames to file 2001/22500
INFO - Writing frames to file 3001/22500
INFO - Writing frames to file 4001/22500
INFO - Writing frames to file 5001/22500
INFO - Writing frames to file 6001/22500
INFO - Writing frames to file 7001/22500
INFO - Writing frames to file 8001/22500
INFO - Writing frames to file 9001/22500
INFO - Writing frames to file 10001/22500
INFO - Writing frames to file 11001/22500
INFO - Writing frames to file 12001/22500
INFO - Writing frames to file 13001/22500
INFO - Writing frames to file 14001/22500
INFO - Writing frames to file 15001/22500
INFO - Writing frames to file 16001/22500
INFO - Writing frames to file 17001/22500
INFO - Writing frames to file 18001/22500
INFO - Writing frames to file 19001/22500
INFO - Writing frames to file 20001/22500
INFO - Writing frames to file 21001/22500
INFO - Writing frames to file 220

In [ ]:
# List of generated frames paths
msks_paths = sorted(glob(os.path.join(output_png_dir, "*.png")))

# Convert the masks to polygons and save them as a WKT file
masks_to_polygons(
    msks_paths,
    out_dim=tuple(config["data"]["original_vid_dims"]),
    save_path=os.path.join(BASE_DIR,"outputs", MODEL, MODE, str(PERCENTAGE), DATA, "WKT", f"{MODE}_{DATA}.wkt"),
)

INFO - Converting masks to polygons...


INFO - Saved polygons to /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt
